In [1]:
'''
Warning: 1.5 GB download.
Download the bag of words (from Google News) from this link: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing
'''

'\nWarning: 1.5 GB download.\nDownload the bag of words (from Google News) from this link: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing\n'

In [2]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from gensim.models import KeyedVectors
from keras.layers import Flatten
from keras.layers import MaxPooling1D
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from nltk.corpus import stopwords

/home/monster/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
MX_NB_WORDS = 200000
MAX_SEQ_LEN = 256
EMBEDDING_DIM = 300

In [4]:
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)

In [5]:
# Peek on the data
print('Data: \t', "\n---------------------------------------------------\n".join(twenty_train.data[:2]))

Data: 	 From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





---------------------------------------------------
From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: Unive

In [6]:
[(idx, val) for idx, val in enumerate(twenty_train.target_names)]

[(0, 'alt.atheism'),
 (1, 'comp.graphics'),
 (2, 'comp.os.ms-windows.misc'),
 (3, 'comp.sys.ibm.pc.hardware'),
 (4, 'comp.sys.mac.hardware'),
 (5, 'comp.windows.x'),
 (6, 'misc.forsale'),
 (7, 'rec.autos'),
 (8, 'rec.motorcycles'),
 (9, 'rec.sport.baseball'),
 (10, 'rec.sport.hockey'),
 (11, 'sci.crypt'),
 (12, 'sci.electronics'),
 (13, 'sci.med'),
 (14, 'sci.space'),
 (15, 'soc.religion.christian'),
 (16, 'talk.politics.guns'),
 (17, 'talk.politics.mideast'),
 (18, 'talk.politics.misc'),
 (19, 'talk.religion.misc')]

In [7]:
EMBEDDING_FILE = "GoogleNews-vectors-negative300.bin"
category_index = {}
for idx, val in enumerate(twenty_train.target_names):
    category_index[idx] = val 
category_reverse_index = dict((y,x) for (x,y) in category_index.items())
STOPWORDS = set(stopwords.words("english"))

In [8]:
import pandas as pd
def preprocess(text):
    text = text.strip().lower().split()
    text = filter(lambda word: word not in STOPWORDS, text)
    return " ".join(text)

def create_dict(arr): # ramdom name - 'title'
    return {'title': arr}

train_df = pd.DataFrame(create_dict(twenty_train.data))
test_df = pd.DataFrame(create_dict(twenty_test.data))

dataset = [train_df, test_df]

for data in dataset:
    data['title'] = data['title'].apply(preprocess)

In [9]:
all_texts = train_df['title'] # + ' ' + test_df['title'] # to include the ' ' between ending of last and new statment.
all_texts = all_texts.drop_duplicates(keep=False)

tokenizer = Tokenizer(num_words=MX_NB_WORDS)
tokenizer.fit_on_texts(all_texts)

train_sequences = tokenizer.texts_to_sequences(train_df['title'])
test_sequences = tokenizer.texts_to_sequences(test_df['title'])

train_data = pad_sequences(train_sequences, maxlen=MAX_SEQ_LEN)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQ_LEN)

In [10]:
print('Train data: \t', train_data.shape)
print('Test data: \t', test_data.shape)
print('-'*20)
category = twenty_train.target
category = to_categorical(category)
print('Labels:\t', category.shape)

Train data: 	 (11314, 256)
Test data: 	 (7532, 256)
--------------------
Labels:	 (11314, 20)


In [11]:
# train - test split
y_test_cat = to_categorical(twenty_test.target)
X_train = train_data
y_train = category
X_test = test_data
y_test = y_test_cat

In [12]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [13]:
from keras.layers import Embedding
word_index = tokenizer.word_index
nb_words = min(MX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
        
print('NULL word embedding: %d'% np.sum(np.sum(embedding_matrix, axis = 1) == 0))
embedding_layer = Embedding(embedding_matrix.shape[0], # or len(word_index) + 1
                            embedding_matrix.shape[1], # or EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LEN,
                            trainable=False)

NULL word embedding: 91382


In [14]:
from keras.models import Sequential, Model
from keras.layers import  Dense, Activation, LSTM, Input

# Input Size
sequence_input = Input(shape=(MAX_SEQ_LEN,), dtype='int32')
# Input layer
embedding_sequence = embedding_layer(sequence_input)
# Hidden layer - 1
x = LSTM(256, activation = 'tanh', return_sequences=True)(embedding_sequence)
# Hidden layer - 2
x = LSTM(256, activation = 'tanh')(x)
# Output layer
preds = Dense(20, activation = 'softmax')(x)
# Loss function 
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 256, 300)          40232400  
_________________________________________________________________
lstm_1 (LSTM)                (None, 256, 256)          570368    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 20)                5140      
Total params: 41,333,220
Trainable params: 1,100,820
Non-trainable params: 40,232,400
_________________________________________________________________


In [15]:
# # Fit the model - Basic
# model.fit(X_train, y_train, validation_data=(X_test, y_test),
#           epochs=20, batch_size=128)
# score = model.evaluate(X_test, y_test, verbose=0)
# print('Test loss: \t', score[0])
# print('Test Accuracy: \t', score[1])

In [16]:
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.callbacks import ModelCheckpoint

# Training the model and saving the best one!!
batch_size = 128
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
earlystop = EarlyStopping(patience=5)
modelsave = ModelCheckpoint(
    filepath='rnn_model_try_no_2.h5', save_best_only=True, verbose=0)
model.fit(
    X_train, y_train, batch_size=batch_size,
    epochs=30, 
    validation_data=(X_test, y_test),
    callbacks=[annealer, earlystop, modelsave]
)

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss: \t', score[0])
print('Test Accuracy: \t', score[1])

Train on 11314 samples, validate on 7532 samples
Epoch 1/30
11314/11314 [==============================] - 766s 68ms/step - loss: 2.5698 - acc: 0.1682 - val_loss: 2.2001 - val_acc: 0.2475
Epoch 2/30
11314/11314 [==============================] - 753s 67ms/step - loss: 1.9881 - acc: 0.3047 - val_loss: 1.7276 - val_acc: 0.3877
Epoch 3/30
11314/11314 [==============================] - 792s 70ms/step - loss: 1.6302 - acc: 0.4252 - val_loss: 2.0204 - val_acc: 0.3524
Epoch 4/30
11314/11314 [==============================] - 873s 77ms/step - loss: 1.4520 - acc: 0.4902 - val_loss: 1.4553 - val_acc: 0.4679
Epoch 5/30
11314/11314 [==============================] - 841s 74ms/step - loss: 1.3131 - acc: 0.5293 - val_loss: 1.4392 - val_acc: 0.4708
Epoch 6/30
11314/11314 [==============================] - 846s 75ms/step - loss: 1.1928 - acc: 0.5774 - val_loss: 1.3614 - val_acc: 0.5364
Epoch 7/30
11314/11314 [==============================] - 841s 74ms/step - loss: 1.1123 - acc: 0.6054 - val_loss: 1.2

In [17]:
# # CNN Model - irrelevant but hey it works !! :D
# from keras.models import Sequential
# from keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout, Activation
# model = Sequential()
# model.add(embedding_layer)
# model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(250))
# model.add(Dropout(0.2))
# model.add(Activation('relu'))
# model.add(Dense(20))
# model.add(Activation('sigmoid'))
# model.compile(loss='categorical_crossentropy', optimizer = 'rmsprop', metrics=['acc'])
# model.summary()

In [18]:
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=128)
# score = model.evaluate(X_test, y_test, verbose=0)
# print('Test loss: \t', score[0])
# print('Test Accuracy: \t', score[1])

# model.save('my_model_20.h5')

In [19]:
# model_1 = Sequential()
# model_1.add(embedding_layer)
# model_1.add(Dropout(0.2))
# model_1.add(Conv1D(512, 3, padding='same',activation='relu',strides=1))
# model_1.add(Conv1D(256, 3, padding='same',activation='relu',strides=1))
# model_1.add(Conv1D(128, 3, padding='same',activation='relu',strides=1))
# model_1.add(Flatten())
# model_1.add(Dropout(0.2))
# model_1.add(Dense(150,activation='sigmoid'))
# model_1.add(Dropout(0.2))
# model_1.add(Dense(20,activation='sigmoid'))

# model_1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

# model_1.summary()

In [20]:
# model_1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=128)
# score = model.evaluate(X_test, y_test, verbose=0)
# print('Test loss: \t', score[0])
# print('Test Accuracy: \t', score[1])

# model.save('my_model_1_20.h5')

In [21]:
print(category_index)

{0: 'alt.atheism', 1: 'comp.graphics', 2: 'comp.os.ms-windows.misc', 3: 'comp.sys.ibm.pc.hardware', 4: 'comp.sys.mac.hardware', 5: 'comp.windows.x', 6: 'misc.forsale', 7: 'rec.autos', 8: 'rec.motorcycles', 9: 'rec.sport.baseball', 10: 'rec.sport.hockey', 11: 'sci.crypt', 12: 'sci.electronics', 13: 'sci.med', 14: 'sci.space', 15: 'soc.religion.christian', 16: 'talk.politics.guns', 17: 'talk.politics.mideast', 18: 'talk.politics.misc', 19: 'talk.religion.misc'}


In [22]:
example_prediction = open('real_test.txt', 'r').read()
example_prediction = preprocess(example_prediction)
example_sequence = tokenizer.texts_to_sequences([example_prediction])
example_padded_sequence = pad_sequences(example_sequence, maxlen=MAX_SEQ_LEN)
# print("Predicted category: ", category_index[model.predict_classes(example_padded_sequence, verbose=0)[0]])
y_pred = model.predict(example_padded_sequence)
cat_predicted = np.argmax(y_pred,axis=1)

print("Predicted category: ", category_index[cat_predicted[0]])

Predicted category:  sci.space
